In [2]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
sys.path.append(os.getcwd()+'/rasotools-master/')
import rasotools

    34	humidity	atmospheric	surface; upper-air	dew point depression	K	Dew point depression is also called dew point deficit. It is the amount by which the air temperature exceeds its dew point temperature. Dew point temperature is the temperature at which a parcel of air reaches saturation upon being cooled at constant pressure and specific humidity.

    36	humidity	atmospheric	surface; upper-air	dew point temperature	K	Dew point temperature is the temperature at which a parcel of air reaches saturation upon being cooled at constant pressure and specific humidity.

    38	humidity	atmospheric	surface; upper-air	relative humidity	percent	TBD

    39	humidity	atmospheric	surface; upper-air	specific humidity	g kg-1	specific means per unit mass. Specific humidity is the mass fraction of water vapor in (moist) air.



    104	wind	atmospheric	surface; upper-air	eastward wind speed	m s-1	Eastward indicates a vector component which is positive when directed eastward (negative westward). Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.)

    105	wind	atmospheric	surface; upper-air	northward wind speed	m s-1	Northward indicates a vector component which is positive when directed northward (negative southward). Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.)

    106	wind	atmospheric	surface; upper-air	wind from direction	degree	direction from which the wind is blowing Lot 1 uses dd  - WMO abbrev.

    107	wind	atmospheric	surface; upper-air	wind speed	m s-1	Speed is the magnitude of velocity. Wind is defined as a two-dimensional (horizontal) air velocity vector,  with no vertical component. (Vertical motion in the atmosphere has the standard name upward air velocity.) The wind speed is the magnitude of the wind velocity. Lot 1 uses ff  - WMO abbrev.

    137	humidity	atmospheric	surface; upper air	air dewpoint	K	Dewpoint measurement (from profile measurement)

    138	humidity	atmospheric	surface; upper air	relative humidity	1	Relative humidity (from profile measurement)

    139	wind	atmospheric	surface; upper air	eastward wind speed	m s-1	Eastward wind speed (from profile measurement)

    140	wind	atmospheric	surface; upper air	northward wind speed	m s-1	Northward wind speed (from profile measurement)

In [3]:
files = glob.glob('/raid60/scratch/uli/resorted_files_correct/*.nc')

In [4]:
def check_for_missing(file, checkvars: list = [85, 38, 106, 107], convert: bool = False):
    missing = []
    with eua.CDMDataset(file) as data:
        for i in checkvars:
            if not str(i) in data.recordindices.keys():
                missing.append(i)
        if not convert:
            return missing
        else: 
            convert_missing(file, missing)

In [5]:
def check_for_missing_or_short(file, checkvars: list = [34, 36, 38, 39, 104, 105, 106, 107], convert: bool = False):
    missing = []
    cleanedvars = checkvars.copy()
    with eua.CDMDataset(file) as data:
        lenlist = []
        for i in checkvars:
            try:
                lenlist.append(len(data.recordindices[str(i)])) # if not readable - variable is missing
            except:
                missing.append(i)
                print(cleanedvars, i)
                cleanedvars.remove(i)
        if len(lenlist) != 0:
            lenmax = max(lenlist)
            for i in range(len(cleanedvars)):
                if lenlist[i] < lenmax: # if one of the variables is shorter than the others
                    missing.append(cleanedvars[i])
        if not convert:
            return missing
        else: 
            convert_missing(file, missing)

In [21]:
#
# todo: 
# change flags
# set fb values? 
# adjust recordindex with addedvarscount
# sort
# write
#


def convert_missing(fn, destination: str = './converted/'):
    with eua.CDMDataset(fn) as data:
        arrayconverter = data.to_dataframe(groups='observations_table', variables=['observed_variable'])
        arrayconverter = arrayconverter.observed_variable.head(1).to_xarray()
        rt = data.recordtimestamp[:]
        keys = data.observations_table.keys()
        keys = [x for x in keys if not x.startswith('string')]
        keys.remove('index')
        keys.remove('shape')
        obskeys = keys
        obstab_writetofile = [[] for i in range(len(obskeys))]
        
        keys = data.era5fb.keys()
        keys = [x for x in keys if not x.startswith('string')]
        keys.remove('index')
        keys.remove('shape')
        fbkeys = keys
        fb_writetofile = [[] for i in range(len(fbkeys))]
        
        recidxlen = len(data.recordindex[:])
        
        addtorecordindex = [] # will be filled with the count of how many variables have been added before !! addtorecordindex[0] has to be added to recordindex [1] !!
        addedvarscount = 0 # will grow with every variable added
        
        for i in range(recidxlen-1):
            obstab = [[]]*len(obskeys)
            fb = [[]]*len(fbkeys)
                   
            for o in range(len(obskeys)):
                obstab[o] = np.asarray(data.observations_table[obskeys[o]][data.recordindex[i]:data.recordindex[i+1]])
                if obskeys[o] == 'observed_variable':
                    obsvar = obstab[o]
                elif obskeys[o] == 'z_coordinate':
                    plev = obstab[o]
                elif obskeys[o] == 'observation_value':
                    obsval = obstab[o]
                    
            for o in range(len(fbkeys)):
                fb[o] = np.asarray(data.era5fb[fbkeys[o]][data.recordindex[i]:data.recordindex[i+1]])
                
            for j in np.unique(plev):
                select = plev == j
                obsvr = obsvar[select]
                obsvl = obsval[select]
                
                for o in range(len(obskeys)):
                    # write everything what was already in the file
                    obstab_writetofile[o].extend(obstab[o][select])
                for o in range(len(fbkeys)):
                    # write everything what was already in the file
                    fb_writetofile[o].extend(fb[o][select])
                    
                #
                # Converting to dewpointdepression
                #
                
                if not (34 in obsvr) and 36 in obsvr:
                    if 85 in obsvr: 
                        t = obsvl[obsvr == 85]
                        dp = obsvl[obsvr == 36]
                        dpd = t-dp

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(34)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(dpd[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_dewpoint'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1

                elif not (34 in obsvr) and 38 in obsvr:
                    if 85 in obsvr: 
                        arrayconverter.values = obsvl[obsvr == 85]
                        t = arrayconverter
                        arrayconverter.values = [j]
                        p = arrayconverter
                        arrayconverter.values = obsvl[obsvr == 38]
                        rh = arrayconverter
                        dpd = rasotools.met.convert.to_dpd(temp=t,press=p,rel_humi=rh)

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(34)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(dpd[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_relative_humidity'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1

                elif not (34 in obsvr) and 39 in obsvr:
                    if 85 in obsvr: 
                        arrayconverter.values = obsvl[obsvr == 85]
                        t = arrayconverter
                        arrayconverter.values = [j]
                        p = arrayconverter
                        arrayconverter.values = obsvl[obsvr == 39]
                        sh = arrayconverter
                        dpd = rasotools.met.convert.to_dpd(temp=t,press=p,spec_humi=sh)

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(34)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(dpd[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_specific_humidity'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1
                        
#                 #
#                 # Converting to dewpoint
#                 #
                
#                 if not (36 in obsvr) and 34 in obsvr:
#                     if 85 in obsvr: 
#                         t = obsvl[obsvr == 85]
#                         dpd = obsvl[obsvr == 34]
#                         dp = t-dp

#                         for o in range(len(obskeys)):
#                             # write the new variable everywhere into the observationstable
#                             if obskeys[o] == 'observed_variable':
#                                 obstab_writetofile[o].append(36)
#                             elif obskeys[o] == 'observation_value':
#                                 obstab_writetofile[o].append(dp[0])
#                             elif obskeys[o] == 'conversion_method':
#                                 obstab_writetofile[o].append(10) # 'from_dewpoint'
#                             elif obskeys[o] == 'observations_id':
#                                 obstab_writetofile[o].append(np.nan)
#                             else:
#                                 obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

#                         for o in range(len(fbkeys)):
#                             # write the new variable everywhere into the fb
#                             # todo: add calculation here
#                             fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
#                         addedvarscount += 1

#                 elif not (36 in obsvr) and 38 in obsvr:
#                     if 85 in obsvr: 
#                         arrayconverter.values = obsvl[obsvr == 85]
#                         t = arrayconverter
#                         arrayconverter.values = [j]
#                         p = arrayconverteraddtorecordindex
#                         arrayconverter.values = obsvl[obsvr == 38]
#                         rh = arrayconverter
#                         dpd = rasotools.met.convert.to_dpd(temp=t,press=p,rel_humi=rh)
#                         dp = t[0] - dpd[0]

#                         for o in range(len(obskeys)):
#                             # write the new variable everywhere into the observationstable
#                             if obskeys[o] == 'observed_variable':
#                                 obstab_writetofile[o].append(36)
#                             elif obskeys[o] == 'observation_value':
#                                 obstab_writetofile[o].append(dp)
#                             elif obskeys[o] == 'conversion_method':
#                                 obstab_writetofile[o].append(10) # 'from_relative_humidity'
#                             elif obskeys[o] == 'observations_id':
#                                 obstab_writetofile[o].append(np.nan)
#                             else:
#                                 obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

#                         for o in range(len(fbkeys)):
#                             # write the new variable everywhere into the fb
#                             # todo: add calculation here
#                             fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
#                         addedvarscount += 1

#                 elif not (36 in obsvr) and 39 in obsvr:
#                     if 85 in obsvr: 
#                         arrayconverter.values = obsvl[obsvr == 85]
#                         t = arrayconverter
#                         arrayconverter.values = [j]
#                         p = arrayconverter
#                         arrayconverter.values = obsvl[obsvr == 39]
#                         sh = arrayconverter
#                         dpd = rasotools.met.convert.to_dpd(temp=t,press=p,spec_humi=sh)
#                         dp = t[0] - dpd[0]

#                         for o in range(len(obskeys)):
#                             # write the new variable everywhere into the observationstable
#                             if obskeys[o] == 'observed_variable':
#                                 obstab_writetofile[o].append(36)
#                             elif obskeys[o] == 'observation_value':
#                                 obstab_writetofile[o].append(dp)
#                             elif obskeys[o] == 'conversion_method':
#                                 obstab_writetofile[o].append(10) # 'from_specific_humidity'
#                             elif obskeys[o] == 'observations_id':
#                                 obstab_writetofile[o].append(np.nan)
#                             else:
#                                 obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

#                         for o in range(len(fbkeys)):
#                             # write the new variable everywhere into the fb
#                             # todo: add calculation here
#                             fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
#                         addedvarscount += 1
                        
                #
                # Converting to relative humidity
                #

                if not (38 in obsvr) and 36 in obsvr:
                    if 85 in obsvr: 
                        t = obsvl[obsvr == 85]
                        dp = obsvl[obsvr == 36]
                        dpd = t-dp
                        arrayconverter.values = dpd
                        dpd = arrayconverter
                        arrayconverter.values = t
                        t = arrayconverter
                        rh = rasotools.met.convert.to_rh(t,dpd=dpd)

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(38)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(rh[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_dewpoint'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1                        

                elif not (38 in obsvr) and 34 in obsvr:
                    if 85 in obsvr: 
                        arrayconverter.values = obsvl[obsvr == 85]
                        t = arrayconverter
                        arrayconverter.values = obsvl[obsvr == 34]
                        dpd = arrayconverter
                        rh = rasotools.met.convert.to_rh(t, dpd=dpd)

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(38)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(rh[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_dewpointdepression'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1

                elif not (38 in obsvr) and 39 in obsvr:
                    if 85 in obsvr: 
                        arrayconverter.values = obsvl[obsvr == 85]
                        t = arrayconverter
                        arrayconverter.values = [j]
                        p = arrayconverter
                        arrayconverter.values = obsvl[obsvr == 39]
                        sh = arrayconverter
                        rh = rasotools.met.convert.to_rh(temp=t, spec_humi=sh, press=p)

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(38)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(rh[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_specific_humidity'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1
                        
                #
                # Converting to specific humidity
                #

                if not (39 in obsvr) and 38 in obsvr:
                    if 85 in obsvr: 
                        arrayconverter.values = obsvl[obsvr == 85]
                        t = arrayconverter
                        arrayconverter.values = [j]
                        p = arrayconverter
                        arrayconverter.values = obsvl[obsvr == 38]
                        rh = arrayconverter
                        sh = rasotools.met.convert.to_sh(temp=t, press=p, rel_humi=rh)

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(39)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(sh[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_relative_humidity'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1                        

                elif not (39 in obsvr) and 34 in obsvr:
                    if 85 in obsvr: 
                        arrayconverter.values = obsvl[obsvr == 85]
                        t = arrayconverter
                        arrayconverter.values = obsvl[obsvr == 34]
                        dpd = arrayconverter
                        arrayconverter.values = [j]
                        p = arrayconverter
                        sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(39)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(rh[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_dewpointdepression'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1

                elif not (39 in obsvr) and 36 in obsvr:
                    if 85 in obsvr: 
                        t = obsvl[obsvr == 85]
                        dp = obsvl[obsvr == 36]
                        arrayconverter.values = t-dp
                        dpd = arrayconverter
                        arrayconverter.values = t
                        t = arrayconverter
                        arrayconverter.values = [j]
                        p = arrayconverter
                        sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)

                        for o in range(len(obskeys)):
                            # write the new variable everywhere into the observationstable
                            if obskeys[o] == 'observed_variable':
                                obstab_writetofile[o].append(39)
                            elif obskeys[o] == 'observation_value':
                                obstab_writetofile[o].append(rh[0])
                            elif obskeys[o] == 'conversion_method':
                                obstab_writetofile[o].append(10) # 'from_dewpoint'
                            elif obskeys[o] == 'observations_id':
                                obstab_writetofile[o].append(np.nan)
                            else:
                                obstab_writetofile[o].append(obstab[o][select][obsvr == 85][0])

                        for o in range(len(fbkeys)):
                            # write the new variable everywhere into the fb
                            # todo: add calculation here
                            fb_writetofile[o].append(fb[o][select][obsvr == 85][0])
                        addedvarscount += 1
                        
                #
                # Converting to wind components
                #
                
                if not (104 in obsvr) or not (105 in obsvr):
                    if 106 in obsvr and 107 in obsvr:
                        wd = obsvl[obsvr == 106]
                        ws = obsvl[obsvr == 107]
                        u = ws * np.cos(np.radians(wd))
                        v = ws * np.sin(np.radians(wd))
                        
                        if not (104 in obsvr):
                            for o in range(len(obskeys)):
                                # write the new variable everywhere into the observationstable
                                if obskeys[o] == 'observed_variable':
                                    obstab_writetofile[o].append(104)
                                elif obskeys[o] == 'observation_value':
                                    obstab_writetofile[o].append(u[0])
                                elif obskeys[o] == 'conversion_method':
                                    obstab_writetofile[o].append(10) # 'from_speed_and_direction'
                                elif obskeys[o] == 'observations_id':
                                    obstab_writetofile[o].append(np.nan)
                                else:
                                    obstab_writetofile[o].append(obstab[o][select][obsvr == 106][0])

                            for o in range(len(fbkeys)):
                                # write the new variable everywhere into the fb
                                # todo: add calculation here
                                fb_writetofile[o].append(fb[o][select][obsvr == 106][0])
                            addedvarscount += 1
                            
                        if not (105 in obsvr):
                            for o in range(len(obskeys)):
                                # write the new variable everywhere into the observationstable
                                if obskeys[o] == 'observed_variable':
                                    obstab_writetofile[o].append(105)
                                elif obskeys[o] == 'observation_value':
                                    obstab_writetofile[o].append(v[0])
                                elif obskeys[o] == 'conversion_method':
                                    obstab_writetofile[o].append(10) # 'from_speed_and_direction'
                                elif obskeys[o] == 'observations_id':
                                    obstab_writetofile[o].append(np.nan)
                                else:
                                    obstab_writetofile[o].append(obstab[o][select][obsvr == 106][0])

                            for o in range(len(fbkeys)):
                                # write the new variable everywhere into the fb
                                # todo: add calculation here
                                fb_writetofile[o].append(fb[o][select][obsvr == 106][0])
                            addedvarscount += 1
                                
                #
                # Converting to windspeed and winddirection
                #
                    
                if not (106 in obsvr) or not (107 in obsvr):
                    if 104 in obsvr and 105 in obsvr:
                        u = obsvl[obsvr == 104][0]
                        v = obsvl[obsvr == 105][0]
                        ws = np.sqrt(u ** 2 + v ** 2)
                        wd = 90 - np.arctan2(-v, -u) * 180 / np.pi - 180.
                        wd = np.where(wd > 0., wd, 360.+wd)
                        
                        if not (106 in obsvr):
                            for o in range(len(obskeys)):
                                # write the new variable everywhere into the observationstable
                                if obskeys[o] == 'observed_variable':
                                    obstab_writetofile[o].append(106)
                                elif obskeys[o] == 'observation_value':
                                    obstab_writetofile[o].append(wd)
                                elif obskeys[o] == 'conversion_method':
                                    obstab_writetofile[o].append(10) # 'from_wind_components'
                                elif obskeys[o] == 'observations_id':
                                    obstab_writetofile[o].append(np.nan)
                                else:
                                    obstab_writetofile[o].append(obstab[o][select][obsvr == 104][0])

                            for o in range(len(fbkeys)):
                                # write the new variable everywhere into the fb
                                # todo: add calculation here
                                fb_writetofile[o].append(fb[o][select][obsvr == 104][0])
                            addedvarscount += 1
                            
                        if not (107 in obsvr):
                            for o in range(len(obskeys)):
                                # write the new variable everywhere into the observationstable
                                if obskeys[o] == 'observed_variable':
                                    obstab_writetofile[o].append(107)
                                elif obskeys[o] == 'observation_value':
                                    obstab_writetofile[o].append(ws)
                                elif obskeys[o] == 'conversion_method':
                                    obstab_writetofile[o].append(10) # 'from_wind_components'
                                elif obskeys[o] == 'observations_id':
                                    obstab_writetofile[o].append(np.nan)
                                else:
                                    obstab_writetofile[o].append(obstab[o][select][obsvr == 104][0])

                            for o in range(len(fbkeys)):
                                # write the new variable everywhere into the fb
                                # todo: add calculation here
                                fb_writetofile[o].append(fb[o][select][obsvr == 104][0])
                            addedvarscount += 1
                                    
            # adjusting the recordindex
            addtorecordindex.append(addedvarscount)
        ri = data.recordindex[:]
        ri = np.asarray([0] + (addtorecordindex)) + np.asarray(ri) # [0] before the array!
            
    # sorting:
    print('start sorting')
    targetfile = destination+fn.split('/')[-1]  
    
    with h5py.File(fn, 'r') as file:
        with h5py.File(targetfile, 'w') as newfile:
            groups = []
            for i in file.keys():
                if type(file[i]) == h5py._hl.group.Group:
                    newfile.create_group(i)
                    groups.append(i)
                elif i == 'recordindex' or i == 'recordtimestamp':
                    pass
                else:
                    newfile.create_dataset(i, data=file[i][:])
            for i in groups:
                if(i == 'recordindices' or i == 'observations_table' or i == 'era5fb'):
                    pass
                else:
                    for j in file[i].keys():
                        newfile[i].create_dataset(j, data=file[i][j][:])
    
#     data =  eua.CDMDataset(fn)
    allvars = np.asarray(obstab_writetofile[obskeys.index('observed_variable')])
    allvars.sort()
    allvars = numpy.unique(allvars)
    #
#     ri = data.recordindex[()]
#     print('recordindex: ', len(ri))
#     rt = data.recordtimestamp[()]
    keys = obskeys # data.observations_table.keys()[:-1]
    fbkeys = fbkeys # data.era5fb.keys()[:-1]
    # dropping all keys, where dimensions won't work - just help variabels for dimensions
    pops = []
    for i in range(len(keys)):
        if 'string' in keys[i]:
            pops.append(keys[i])
    for i in pops: keys.remove(i)
    pops = []
    for i in range(len(fbkeys)):
        if 'string' in fbkeys[i]:
            pops.append(fbkeys[i])
    for i in pops: fbkeys.remove(i)

    recordindices = [[] for i in range(len(allvars))]
    recordtimestamps = [[] for i in range(len(allvars))]

    # output variables (from observations_table)
    ov = []
    for o in keys:
        ov.append([[] for i in range(len(allvars))])
    fb = []
    for o in fbkeys:
        fb.append([[] for i in range(len(allvars))])
    #
    # loading the observed_variables
    #
    obsv = np.asarray(obstab_writetofile[obskeys.index('observed_variable')]) # data.observations_table.observed_variable[:]
    #
    # resorting the data
    #
#     print('resort:start')
    @njit
    def make_vrindex(vridx,ridx,idx):
        l=0
        for i in range(1,len(idx)): # to set the recordindices
            if ridx[i]>ridx[i-1]:
                vridx[ridx[i-1]:ridx[i]]=l # next record after l
                l=i
        vridx[ridx[i]:]=len(idx) # next record for the last element is the len of the data


    tt=time.time()

    ridxall=np.zeros(obsv.shape[0],dtype=np.int64) # reverse index - index of the record index
    j=-1
    for j in range(len(ri)-1):
        ridxall[ri[j]:ri[j+1]]=j
    j+=1
    ridxall[ri[j]:]=j # for the last elemenet
    ridx=[]
    vridx=[]
    absidx=[]
    abscount=0
    for j in range(len(allvars)):
        idx=np.where(obsv==allvars[j])[0] # index of all elements form certain variable j
#         print(j,len(idx),',',end='')
        vridx.append(np.zeros(ri.shape[0],dtype=np.int64)) # all zeros in lenght of record index
        ridx=ridxall[idx] # ridxall where variable is j
        make_vrindex(vridx[-1],ridx,idx)
        vridx[-1]+=abscount # abscount for stacking the recordindex

        absidx.append(copy.copy(idx)) # why copy? - to make sure it's not just the ref. - maybe ok without the cp
        abscount+=len(idx)

    # finishing the sorting 
    #
    absidx=np.concatenate(absidx)
    #
    # recordtimestamps are only necessary once
    #
    recordtimestamps = recordtimestamps[0]
    #
    # targetfile has to be a copy of the original file
    #
    
    if os.path.isfile(targetfile):
        mode='r+'
    else:
        mode='w'
#     print()
#     print('writing '+targetfile)
    
    for i in range(len(keys)):
        ov_vars = np.asarray(obstab_writetofile[i]) # data.observations_table[keys[i]][:]
        ov_vars = ov_vars[absidx]
        if keys[i] == 'index':
            pass
        elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'source_id':
            alldict = {keys[i]:np.asarray(ov_vars, dtype='S1')}
            write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
        else:
            alldict = pandas.DataFrame({keys[i]:ov_vars})
            write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  

    for i in range(len(fbkeys)):
        fb_vars = np.asarray(fb_writetofile[i]) # data.era5fb[fbkeys[i]][:]
        fb_vars = fb_vars[absidx]
        if fbkeys[i] == 'index' or fbkeys[i] == 'string6' or fbkeys[i] == 'string7' or fbkeys[i] == 'string10':
            pass
        elif fbkeys[i] == 'expver' or fbkeys[i] == 'source@hdr' or fbkeys[i] == 'source_id' or fbkeys[i] == 'statid@hdr':
            alldict = {fbkeys[i]:np.asarray(fb_vars, dtype='S1')}
            write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]])
        else:
            alldict = pandas.DataFrame({fbkeys[i]:fb_vars})
            write_dict_h5(targetfile, alldict, 'era5fb', {fbkeys[i]: { 'compression': 'gzip' } }, [fbkeys[i]]) 
    #
    # writing the recordindices and recordtimestamp.
    #       
    recordindices=vridx
    for i in range(len(recordindices)):
        testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
        write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': None } }, [str(allvars[i])]) 

    write_dict_h5(targetfile, {'recordtimestamp':rt}, 'recordindices', {'recordtimestamp': { 'compression': None } }, ['recordtimestamp'])

    print('elapsed:',time.time()-tt)


In [22]:
convert_missing('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc')

/jetfs/home/uvoggenberger/CEUAS/CEUAS/public/resort/rasotools-master/rasotools/met/esat.py:439: RuntimeWarning: divide by zero encountered in true_divide
  -5800.2206 / t + 1.3914993
/jetfs/home/uvoggenberger/CEUAS/CEUAS/public/resort/rasotools-master/rasotools/met/esat.py:443: RuntimeWarning: divide by zero encountered in log
  + 6.5459673 * np.log(t)


start sorting
elapsed: 1.5167105197906494


In [24]:
filesout = glob.glob('converted/*.nc')
filesout

['converted/0-20000-0-11019_CEUAS_merged_v0.nc']

In [31]:
with eua.CDMDataset(filesout[0]) as test:
    print(test.observations_table.observed_variable[:])

[ 34  34  34 ... 140 140 140]


In [ ]:
for i in range(len(add)):
    ri[i+1] = ri[i+1] + add[i]

In [22]:
np.asarray([0] + (add)) + np.asarray(ri)

array([    0,   119,   306,   527,   604,   727,   874,   989,  1090,
        1191,  1340,  1467,  1580,  1727,  1820,  1955,  2100,  2317,
        2374,  2495,  2638,  2775,  2908,  3059,  3196,  3333,  3432,
        3571,  3700,  3877,  4002,  4115,  4280,  4497,  4616,  4753,
        4886,  5083,  5170,  5279,  5414,  5559,  5674,  5801,  5906,
        6045,  6200,  6367,  6522,  6621,  6741,  6886,  7033,  7170,
        7357,  7506,  7663,  7794,  8011,  8162,  8373,  8538,  8715,
        8872,  9009,  9144,  9217,  9306,  9419,  9546,  9641,  9752,
        9903, 10014, 10111, 10202, 10289, 10430, 10575, 10762, 10941,
       11054, 11163, 11280, 11517, 11668, 11835, 11934, 12061, 12268,
       12435, 12578, 12733, 12910, 13033, 13156, 13327, 13410, 13569,
       13716, 13813, 13878, 13949, 14088, 14211, 14278, 14405, 14560,
       14711, 14898, 15045, 15262, 15313, 15406, 15503, 15720, 15875,
       16048, 16229, 16346, 16475, 16612, 16805, 17016, 17217, 17368,
       17575, 17734,

In [18]:
np.asarray(ri)

array([    0,    99,   266,   464,   530,   633,   764,   863,   951,
        1039,  1169,  1282,  1376,  1507,  1587,  1704,  1828,  2022,
        2072,  2180,  2303,  2425,  2537,  2672,  2794,  2916,  2999,
        3116,  3226,  3384,  3494,  3592,  3732,  3926,  4025,  4141,
        4257,  4433,  4506,  4598,  4711,  4835,  4932,  5039,  5127,
        5244,  5375,  5524,  5659,  5742,  5846,  5972,  6103,  6223,
        6390,  6520,  6658,  6769,  6963,  7092,  7281,  7423,  7581,
        7715,  7831,  7946,  8010,  8088,  8186,  8293,  8376,  8473,
        8608,  8703,  8787,  8864,  8939,  9065,  9191,  9358,  9515,
        9613,  9705,  9807, 10019, 10154, 10299, 10382, 10489, 10674,
       10815, 10940, 11075, 11229, 11332, 11437, 11588, 11661, 11800,
       11929, 12013, 12069, 12130, 12249, 12356, 12415, 12524, 12659,
       12786, 12953, 13080, 13274, 13317, 13397, 13479, 13673, 13810,
       13964, 14126, 14228, 14340, 14460, 14628, 14817, 14997, 15124,
       15309, 15448,

In [21]:
np.asarray([0] + (add))

array([   0,   20,   40,   63,   74,   94,  110,  126,  139,  152,  171,
        185,  204,  220,  233,  251,  272,  295,  302,  315,  335,  350,
        371,  387,  402,  417,  433,  455,  474,  493,  508,  523,  548,
        571,  591,  612,  629,  650,  664,  681,  703,  724,  742,  762,
        779,  801,  825,  843,  863,  879,  895,  914,  930,  947,  967,
        986, 1005, 1025, 1048, 1070, 1092, 1115, 1134, 1157, 1178, 1198,
       1207, 1218, 1233, 1253, 1265, 1279, 1295, 1311, 1324, 1338, 1350,
       1365, 1384, 1404, 1426, 1441, 1458, 1473, 1498, 1514, 1536, 1552,
       1572, 1594, 1620, 1638, 1658, 1681, 1701, 1719, 1739, 1749, 1769,
       1787, 1800, 1809, 1819, 1839, 1855, 1863, 1881, 1901, 1925, 1945,
       1965, 1988, 1996, 2009, 2024, 2047, 2065, 2084, 2103, 2118, 2135,
       2152, 2177, 2199, 2220, 2244, 2266, 2286, 2306, 2326, 2343, 2362,
       2378, 2395, 2412, 2428, 2446, 2463, 2485, 2501, 2517, 2538, 2559,
       2576, 2596, 2618, 2636, 2657, 2674, 2693, 27

In [13]:
%load_ext line_profiler
%lprun -f convert_missing convert_missing('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc')

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


/jetfs/home/uvoggenberger/CEUAS/CEUAS/public/resort/rasotools-master/rasotools/met/esat.py:439: RuntimeWarning: divide by zero encountered in true_divide
  -5800.2206 / t + 1.3914993
/jetfs/home/uvoggenberger/CEUAS/CEUAS/public/resort/rasotools-master/rasotools/met/esat.py:443: RuntimeWarning: divide by zero encountered in log
  + 6.5459673 * np.log(t)


Timer unit: 1e-06 s

Total time: 24.6186 s
File: <ipython-input-11-559361eb4a55>
Function: convert_missing at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def convert_missing(file, destination: str = './converted/'):
    12         1     642949.0 642949.0      2.6      with eua.CDMDataset(file) as data:
    13         1       2484.0   2484.0      0.0          arrayconverter = data.to_dataframe(groups='observations_table', variables=['observed_variable'])
    14         1       2492.0   2492.0      0.0          arrayconverter = arrayconverter.observed_variable.head(1).to_xarray()
    15         1          4.0      4.0      0.0          keys = data.observations_table.keys()
    16         1         18.0     18.0      0.0          keys = [x for x in keys if not x.startswith('string')]
    17         1          3.0      3.0      0.0          keys.remove('index')
    18         1          2.0      2.0      0.0      

In [20]:
ob, fb , ri= convert_missing('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc')

In [27]:
ob[25]

[30750.0,
 272.3,
 249.1,
 0.1527943,
 0.0007797543,
 array([23.199982], dtype=float32),
 29750.0,
 272.5,
 249.2,
 0.15193212,
 0.00077678345,
 array([23.300003], dtype=float32),
 25760.0,
 273.7,
 249.4,
 0.14175875,
 0.0007515748,
 array([24.300018], dtype=float32),
 25340.0,
 273.8,
 250.3,
 0.15247771,
 0.0008099545,
 array([23.499985], dtype=float32),
 20650.0,
 274.8,
 251.1,
 0.15228188,
 0.00081903086,
 array([23.699982], dtype=float32),
 20260.0,
 275.0,
 251.3,
 0.15278015,
 0.0008293987,
 array([23.699997], dtype=float32),
 19280.0,
 275.4,
 251.6,
 0.15243694,
 0.00084100995,
 array([23.799988], dtype=float32),
 15480.0,
 273.0,
 251.7,
 0.182789,
 0.00080846576,
 array([21.300003], dtype=float32),
 14300.0,
 272.2,
 261.8,
 0.45117056,
 0.0018551712,
 array([10.400024], dtype=float32),
 11000.0,
 274.2,
 264.8,
 0.49448735,
 0.0022549613,
 array([9.400024], dtype=float32),
 10130.0,
 274.7,
 265.2,
 0.49211696,
 0.0023007512,
 array([9.5], dtype=float32),
 8250.0,
 272.5,

In [30]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    print(data.recordindex)
    print(data.observations_table)
    print(data.era5fb)

recordindex : (312,)
observations_table:

adjustment_id_____________________________________ : : (41144,)
advanced_assimilation_feedback____________________ : : (41144,)
advanced_homogenisation___________________________ : : (41144,)
advanced_qc_______________________________________ : : (41144,)
advanced_uncertainty______________________________ : : (41144,)
bbox_max_latitude_________________________________ : : (41144,)
bbox_max_longitude________________________________ : : (41144,)
bbox_min_latitude_________________________________ : : (41144,)
bbox_min_longitude________________________________ : : (41144,)
code_table________________________________________ : : (41144,)
conversion_flag___________________________________ : : (41144,)
conversion_method_________________________________ : : (41144,)
crs_______________________________________________ : : (41144,)
data_policy_licence_______________________________ : : (41144,)
date_time_________________________________________ : : (41144,

In [79]:
_74[26][:30]

array([117.,  85.,  36.,  38.,  39.,  34., 117.,  85.,  36.,  38.,  39.,
        34., 117.,  85.,  36.,  38.,  39.,  34., 117.,  85.,  36.,  38.,
        39.,  34., 117.,  85.,  36.,  38.,  39.,  34.])

In [77]:
obskeys.index('observed_variable')

26

In [ ]:
                        obstab_obstab_writetofile[o] = obstab_writetofile[o] + list(obstab[o][select])
                        if obskeys[o] == 'observed_variable':
                            obstab_writetofile[o] = obstab_writetofile[o] + [34]
                        elif obskeys[o] == 'observation_value':
                            obstab_writetofile[o] = obstab_writetofile[o] + [t-dp]
                        elif obskeys[o] == 'conversion_methode':
                            obstab_writetofile[o] = obstab_writetofile[o] +['from_dewpoint']
                        else:
                            obstab_writetofile[o] = obstab_writetofile[o] + list(obstab[o][select][obsvl == 85])

In [125]:
a = [7,8,9,8]
b = [8,1,4,5]
a+b
[34]

[34]

In [126]:
a = convert_missing('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc', [38])
a

DEBUG:upperair.cdm:[OPEN] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc
INFO:upperair.cdm:Reading Profile on slice(None, None, None)
DEBUG:upperair.cdm:Read variables: dict_keys(['date_time', 'observed_variable', 'observation_value', 'z_coordinate'])
DEBUG:upperair.cdm:[CLOSED] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc


[[nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,

In [129]:
obskeys.index('observed_variable')

26

In [137]:
def convert_missing(file, missing, destination: str = './converted/'):
    with eua.CDMDataset(file) as data:
        keys = data.observations_table.keys()
        keys = [x for x in keys if not x.startswith('string')]
        keys.remove('index')
        keys.remove('shape')
        obskeys = keys
        obstab_writetofile = []
        for i in obskeys:
            obstab_writetofile.append([])
        keys = ['date_time', 'observed_variable', 'observation_value', 'z_coordinate']
        df = data.to_dataframe(groups='observations_table', variables=keys)
        obstable = {}
        for i in obskeys:
            obstable[i] = data.observations_table[i][:]
        for i in df.date_time.drop_duplicates():
            da = df[df.date_time == i]
            for j in da.z_coordinate.drop_duplicates():
                dx = da[da.z_coordinate == j] # -> one pressure level and one point in time
                
                # write everything existing into the output list
                start = dx.observation_value.head(1).index[0] 
                end = dx.observation_value.tail(1).index[0]
                for o in range(len(obskeys)):
                    obstab_writetofile[o] = obstab_writetofile[o] + list(obstable[obskeys[o]][start:end])
                    
                # check for missing variables
                obsv = list(dx.observed_variable)
                
                if not (34 in obsv) and 36 in obsv:
                    t = dx[dx.observed_variable == 85]
                    hum = dx[dx.observed_variable == 36]
                    sourceindex = hum.date_time.index[0] # -> we copy the information for the new variable from here
                    
                    for o in range(len(obskeys)):
                        if obskeys[o] == 'observed_variable':
                            obstab_writetofile[o] = obstab_writetofile[o] + [34]
                        elif obskeys[o] == 'observation_value':
                            obstab_writetofile[o] = obstab_writetofile[o] + [t.observation_value - hum.observation_value]
                        elif obskeys[o] == 'conversion_methode':
                            obstab_writetofile[o] = obstab_writetofile[o] + ['from_dewpoint']
                        else:
                            obstab_writetofile[o].append(obstable[obskeys[o]][sourceindex])
#                 elif not (34 in obsv) and 38 in obsv:
#                     print('34 could be converted from 38')
#                 elif not (34 in obsv) and 39 in obsv:
#                     print('34 could be converted from 39')
                # ... all other checks
    return obstab_writetofile
        

In [138]:
%load_ext line_profiler
%lprun -f convert_missing convert_missing('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc', [38])

DEBUG:upperair.cdm:[OPEN] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc
INFO:upperair.cdm:Reading Profile on slice(None, None, None)
DEBUG:upperair.cdm:Read variables: dict_keys(['date_time', 'observed_variable', 'observation_value', 'z_coordinate'])


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


DEBUG:upperair.cdm:[CLOSED] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc


Timer unit: 1e-06 s

Total time: 121.023 s
File: <ipython-input-137-f9e439eab3c6>
Function: convert_missing at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def convert_missing(file, missing, destination: str = './converted/'):
     2         1     186776.0 186776.0      0.2      with eua.CDMDataset(file) as data:
     3         1          4.0      4.0      0.0          keys = data.observations_table.keys()
     4         1         23.0     23.0      0.0          keys = [x for x in keys if not x.startswith('string')]
     5         1          3.0      3.0      0.0          keys.remove('index')
     6         1          1.0      1.0      0.0          keys.remove('shape')
     7         1          1.0      1.0      0.0          obskeys = keys
     8         1          1.0      1.0      0.0          writetofile = []
     9        47         34.0      0.7      0.0          for i in obskeys:
    10        46         4

In [130]:
a[26]

[117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 34,
 117,
 136,
 137,
 138,
 39,
 106,
 107,
 139,
 117,
 85,
 36,
 38,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 38,
 39,
 106,
 107,
 104,
 34,
 117,
 85,
 36,
 3

In [98]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    keys = ['date_time', 'observed_variable', 'observation_value', 'z_coordinate']
    df = data.to_dataframe(groups='observations_table', variables=keys)
    print(df.observed_variable.tail(1))
    print(df.observation_value.tail(1))
    print(df.observation_value.head(1).index[0])
    print(df.observation_value.tail(1).index[0])
    print(data.observations_table.observation_value[41143])

DEBUG:upperair.cdm:[OPEN] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc
INFO:upperair.cdm:Reading Profile on slice(None, None, None)
DEBUG:upperair.cdm:Read variables: dict_keys(['date_time', 'observed_variable', 'observation_value', 'z_coordinate'])
DEBUG:upperair.cdm:[CLOSED] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc


41143    140
Name: observed_variable, dtype: int64
41143    2.954423
Name: observation_value, dtype: float32
0
41143
2.9544232


In [84]:
with eua.CDMDataset('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    keys = data.observations_table.keys()
    keys = [x for x in keys if not x.startswith('string')]
    keys.remove('index')
    keys.remove('shape')
    obskeys = keys
    obstab_writetofile = []
    for i in obskeys:
        obstab_writetofile.append([])
    print(obstab_writetofile)

DEBUG:upperair.cdm:[OPEN] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc
DEBUG:upperair.cdm:[CLOSED] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc


[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [77]:
convert_missing('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc', [38])

DEBUG:upperair.cdm:[OPEN] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc
INFO:upperair.cdm:Reading Profile on slice(None, None, None)
DEBUG:upperair.cdm:Read variables: dict_keys(['date_time', 'observed_variable', 'observation_value', 'z_coordinate'])
DEBUG:upperair.cdm:[CLOSED] /raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11019_CEUAS_merged_v0.nc


            date_time  observed_variable  observation_value  z_coordinate
1 1972-10-30 14:00:00                 34          23.199982       70000.0
1    272.299988
Name: observation_value, dtype: float32
272.3
            date_time  observed_variable  observation_value  z_coordinate
6 1972-10-30 14:00:00                 34          23.300003       70900.0
6    272.5
Name: observation_value, dtype: float32
272.5
             date_time  observed_variable  observation_value  z_coordinate
11 1972-10-30 14:00:00                 34          24.300018       74600.0
11    273.700012
Name: observation_value, dtype: float32
273.7
             date_time  observed_variable  observation_value  z_coordinate
16 1972-10-30 14:00:00                 34          23.499985       75000.0
16    273.799988
Name: observation_value, dtype: float32
273.8
             date_time  observed_variable  observation_value  z_coordinate
21 1972-10-30 14:00:00                 34          23.699982       79600.0
21    274

NameError: name 'from36' is not defined

In [5]:
# einfach dazu schreiben und dann neu sortieren? wäre eine option
# schreiben, era5fb und recordinidce anpassen
# flags setzten - siehe rückgabe convert
# vermerken, wenn etwas nicht passt - siehe rückgabe convert
#



def convert_missing(file, missing, destination: str = './converted/'):
    towrite = []
    with eua.CDMDataset(file) as data:
        convertible = ['104', '105', '106', '107', '34', '36', '38', '39']
        missingrecids = []
        for i in convertible:
            if not i in data.recordindices.keys():
                missingrecids.append(i)
        print('loading data')
#         df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'z_coordinate', 'report_id'])
        keys = data.observations_table.keys()
        keys = [x for x in keys if not x.startswith('string')]
        keys.remove('index')
        keys.remove('shape')
        df = data.to_dataframe(groups='observations_table', variables=keys)

        print('loading complete')

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 38 in missing: # rh is missing -> calculate it from 
        print('converting to 38')
        hums = [34, 36, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
        print(hums)
        rhall = []
        if 34 in hums: # dew point depression
            print('with 34')
            with eua.CDMDataset(file) as data:
                t = data.observations_table.observation_value
                t = t[data.observations_table.observed_variable == 85]
            hum = df[df.observed_variable == 34]
            
            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.d.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])
        
            t = cleaned_t.observation_value.to_xarray()
            dpd = cleaned_hum.observation_value.to_xarray()
            
            rh = rasotools.met.convert.to_rh(t, dpd=dpd)
            out = cleaned_t.copy()
            out['observed_variable'] =  38 
            out['observation_value'] =  rh
            rhall.append(out)
            #add
            #set conversion flag
        if 36 in hums: # dew point
            print('with 36')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 36]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            dpd = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))
            t = cleaned_t.observation_value.to_xarray()
            dpd = dpd.to_xarray()

            rh = rasotools.met.convert.to_rh(t,dpd=dpd)  
            out = cleaned_t.copy()
            out['observed_variable'] =  38 
            out['observation_value'] =  rh
            rhall.append(out)
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            print('with 39')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 39]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            p = cleaned_t.z_coordinate.to_xarray()
            t = cleaned_t.observation_value.to_xarray()
            sh = cleaned_hum.observation_value.to_xarray()

            rh = rasotools.met.convert.to_rh(temp=t, spec_humi=sh, press=p)
            out = cleaned_t.copy()
            out['observed_variable'] =  38 
            out['observation_value'] =  rh
            rhall.append(out)

            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
            
        comb = rhall[0][0:0]
        for i in range(len(rhall)):
            comb = comb.append(rhall[i], ignore_index=True)
        comb = df[df.observed_variable == 38].append(comb)
        # drop_duplicates with keep='first' -> values, which were already available stay the same
        comb = comb.drop_duplicates(('report_id', 'z_coordinate'))
        towrite.append(comb)   
        
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 34 in missing: # dew point depression is missing -> calculate it from
        print('converting to 34')
        hums = [38, 36, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
        print(hums)
        dpdall = []
        if 38 in hums and not 85 in missing: # relative humidity
            print('with 38')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 38]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            rh = cleaned_hum.observation_value.to_xarray()

            dpd = rasotools.met.convert.to_dpd(temp=t,press=p,rel_humi=rh)
            out = cleaned_t.copy()
            out['observed_variable'] =  34
            out['observation_value'] =  dpd
            dpdall.append(out)
            #add
            #set conversion flag
        if 36 in hums: # dew point
            print('with 36')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 36]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])


            dpd = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))
            dpd = dpd.to_xarray()
            out = cleaned_t.copy()
            out['observed_variable'] =  34
            out['observation_value'] =  dpd
            dpdall.append(out)
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            print('with 39')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 39]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])
                
            p = cleaned_t.z_coordinate.to_xarray()
            t = cleaned_t.observation_value.to_xarray()
            sh = cleaned_hum.observation_value.to_xarray()
            print(p.values)

            dpd = rasotools.met.convert.to_dpd(spec_humi=sh,press=p,temp=t)
            out = cleaned_t.copy()
            out['observed_variable'] =  34
            out['observation_value'] =  dpd
            dpdall.append(out)
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
        comb = dpdall[0][0:0]
        for i in range(len(dpdall)):
            comb = comb.append(dpdall[i], ignore_index=True)
        comb = comb.drop_duplicates(('report_id', 'z_coordinate'))
        towrite.append(comb)
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 36 in missing: # dew point is missing -> calculate it from
        print('converting to 36')
        hums = [34, 38, 39]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
        print(hums)
        dpall = []
        if 38 in hums and not 85 in missing: # relative humidity
            print('with 38')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 38]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            rh = cleaned_hum.observation_value.to_xarray()

            dp = rasotools.met.convert.to_dewpoint(temp=t,press=p,rel_humi=rh)
            out = cleaned_t.copy()
            out['observed_variable'] =  36
            out['observation_value'] =  dp
            dpall.append(out)
            #add
            #set conversion flag
        if 34 in hums: # dew point depression
            print('with 34')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 34]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            dp = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))
            dp = dp.to_xarray()
            out = cleaned_t.copy()
            out['observed_variable'] =  36
            out['observation_value'] =  dp
            dpall.append(out)
            #add
            #set conversion flag
        if 39 in hums: # specific humidity
            print('with 39')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 39]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            p = cleaned_t.z_coordinate.to_xarray()
            sh = cleaned_hum.observation_value.to_xarray()
            dp = rasotools.met.convert.to_dewpoint(spec_humi=sh,press=p)
            out = cleaned_t.copy()
            out['observed_variable'] =  36
            out['observation_value'] =  dp
            dpall.append(out)
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
        comb = dpall[0][0:0]
        for i in range(len(dpall)):
            comb = comb.append(dpall[i], ignore_index=True)
        comb = comb.drop_duplicates(('report_id', 'z_coordinate'))
        towrite.append(comb)
            
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 39 in missing: # specific humidity is missing -> calculate it from
        print('converting to 39')
        hums = [34, 38, 36]
        missing_hum_vars = check_for_missing(file, checkvars = hums)
        for i in missing_hum_vars: hums.remove(i)
        print(hums)
        shall = []
        if 38 in hums and not 85 in missing: # relative humidity
            print('with 38')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 38]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            rh = cleaned_hum.observation_value.to_xarray()

            sh = rasotools.met.convert.to_sh(temp=t, press=p, rel_humi=rh)
            out = cleaned_t.copy()
            out['observed_variable'] =  39
            out['observation_value'] =  sh
            shall.append(out)
            #add
            #set conversion flag
        if 34 in hums: # dew point depression
            print('with 34')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 34]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            dpd = cleaned_hum.observation_value.to_xarray()

            sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
            out = cleaned_t.copy()
            out['observed_variable'] =  39
            out['observation_value'] =  sh
            shall.append(out)
            #add
            #set conversion flag
        if 36 in hums: # dew point
            print('with 36')
            t = df[df.observed_variable == 85]
            hum = df[df.observed_variable == 36]

            cleaned_t = t[0:0]
            cleaned_hum = hum[0:0]
            for i in hum.report_id.drop_duplicates():
                inter_t = t[t.report_id == i]
                inter_hum = hum[hum.report_id == i]
                cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
                cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

            dpd = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))

            t = cleaned_t.observation_value.to_xarray()
            p = cleaned_t.z_coordinate.to_xarray()
            dpd = dpd.to_xarray()
            sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
            out = cleaned_t.copy()
            out['observed_variable'] =  39
            out['observation_value'] =  sh
            shall.append(out)
            #add
            #set conversion flag
        else:
            print('-')
            print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
            print('-')
        comb = shall[0][0:0]
        for i in range(len(shall)):
            comb = comb.append(shall[i], ignore_index=True)
        comb = comb.drop_duplicates(('report_id', 'z_coordinate'))
        towrite.append(comb)
    
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

    if 104 in missing or 105 in missing: # u and/or v is missing -> calculate it from wd and ws
        wd = df[df.observed_variable == 106]
        ws = df[df.observed_variable == 107]
        
        if(len(wd) != len(ws)):
            print('not of equal length')
            cleaned_wd = wd[0:0]
            cleaned_ws = ws[0:0]
            for i in ws.report_id.drop_duplicates():
                inter_wd = wd[wd.report_id == i]
                inter_ws = ws[ws.report_id == i]
                cleaned_wd = cleaned_wd.append(inter_wd[inter_wd.z_coordinate.isin(inter_ws.z_coordinate)])
                cleaned_ws = cleaned_ws.append(inter_ws[inter_ws.z_coordinate.isin(inter_wd.z_coordinate)])
            print('equal length - start processing')
            wd = np.asarray(cleaned_wd.observation_value)
            ws = np.asarray(cleaned_ws.observation_value)
            out_u = cleaned_wd.copy()
            out_v = cleaned_wd.copy()

        else:
            print('equal length - start processing')
            out_u = wd.copy()
            out_v = wd.copy()
            wd = np.asarray(wd.observation_value)
            ws = np.asarray(ws.observation_value)


        u = ws * np.cos(np.radians(wd))
        v = ws * np.sin(np.radians(wd))
        out_u['observed_variable'] =  104 
        out_u['observation_value'] =  u
        out_v['observed_variable'] =  105 
        out_v['observation_value'] =  v
        comb = df[df.observed_variable == 104].append(out_u)
        # drop_duplicates with keep='first' -> values, which were already available stay the same
        comb = comb.drop_duplicates(('report_id', 'z_coordinate'))
        towrite.append(comb)
        comb = df[df.observed_variable == 105].append(out_v)
        # drop_duplicates with keep='first' -> values, which were already available stay the same
        comb = comb.drop_duplicates(('report_id', 'z_coordinate'))
        towrite.append(comb)
        #add
        #set conversion flag
        
    if 106 in missing or 107 in missing: # wd and/or ws is missing -> calculate it from u/v
        u = df[df.observed_variable == 104]
        v = df[df.observed_variable == 105]
        
        if(len(u) != len(v)):
            print('not of equal length')
            cleaned_u = u[0:0]
            cleaned_v = v[0:0]
            for i in v.report_id.drop_duplicates():
                inter_u = u[u.report_id == i]
                inter_v = v[v.report_id == i]
                cleaned_u = cleaned_u.append(inter_u[inter_u.z_coordinate.isin(inter_v.z_coordinate)])
                cleaned_v = cleaned_v.append(inter_v[inter_v.z_coordinate.isin(inter_u.z_coordinate)]) 
            print('equal length - start processing')
            u = np.asarray(cleaned_u.observation_value)
            v = np.asarray(cleaned_v.observation_value)
            out_wd = cleaned_u.copy()
            out_ws = cleaned_u.copy()
        else:
            print('equal length - start processing')
            out_wd = u.copy()
            out_ws = u.copy()
            u = np.asarray(u.observation_value)
            v = np.asarray(v.observation_value)

        ws = np.sqrt(u ** 2 + v ** 2)
        wd = 90 - np.arctan2(-v, -u) * 180 / np.pi - 180.
        wd = np.where(wd > 0., wd, 360.+wd)
        
        out_wd['observed_variable'] =  106 
        out_wd['observation_value'] =  wd
        out_ws['observed_variable'] =  107 
        out_ws['observation_value'] =  ws
        comb = df[df.observed_variable == 106].append(out_wd)
        # drop_duplicates with keep='first' -> values, which were already available stay the same
        comb = comb.drop_duplicates(('report_id', 'z_coordinate'))
        towrite.append(comb)
        comb = df[df.observed_variable == 107].append(out_ws)
        # drop_duplicates with keep='first' -> values, which were already available stay the same
        comb = comb.drop_duplicates(('report_id', 'z_coordinate'))
        towrite.append(comb)
        #add
        #set conversion flag
        
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
   
    return towrite
#     print('creating new file')
#     targetfile = destination + file.split('/')[-1]
#     with h5py.File(file, 'r') as origin:
#         with h5py.File(targetfile, 'w') as newfile:
#             groups = []
#             for i in origin.keys():
#                 if type(origin[i]) == h5py._hl.group.Group:
#                     newfile.create_group(i)
#                     groups.append(i)
#                 else:
#                     newfile.create_dataset(i, data=origin[i][:])
#             for i in groups:
#                 if(i == 'recordindices' or i == 'observations_table' or i == 'era5fb'):
#                     pass
#                 else:
#                     for j in origin[i].keys():
#                         newfile[i].create_dataset(j, data=origin[i][j][:])
                        
#     print('writing converted data to file')
#     with h5py.File(file, 'r') as origin:
#         obs = []
#         for o in towrite:
#             if o.observed_variable.iloc[0] in missingrecids:
#                 # writing observations_table
#                 for i in o.keys():
#                     testvar = pandas.DataFrame({i:origin.observations_table[i][:].append(o[i])}) # puts it right on the end, and not in a numericaly sorted way
#                     write_dict_h5(targetfile, testvar, 'observations_table', {i: { 'compression': 'gzip' } }, [i])
                    
#                 # writing recordindices
#                 # o.observed_variable.iloc[0] is the new recordindex - the number not already in the group
#                 # has to be fitting for recordtimestamp too, 
#                 # iterate over recordtimestamp and check for the frist index, where it occures in the data, if not, select the last/next? one
                
                
#                 np.where(v[:-1] != v[1:])[0]
#                 testvar = pandas.DataFrame({str(o.observed_variable.iloc[0]):recordindices[i]})
#                 write_dict_h5(targetfile, testvar, 'recordindices', {str(o.observed_variable.iloc[0]): { 'compression': 'gzip' } }, [str(o.observed_variable.iloc[0])])
                
                
#                 # writing era5fb

#             else:
                

#         for i in origin.era5fb.keys():
            
#         for i in origin.recordindices.keys():
#             testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
#             write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': 'gzip' } }, [str(allvars[i])])
#         origin 
#     with h5py.File(targetfile, 'w') as newfile:
#         for i in origin['observations_table'].keys():
#             if(i == 'recordindices' or i == 'observations_table' or i == 'era5fb'):
#                 pass
#             else:
#                 for j in origin[i].keys():
#                     newfile[i].create_dataset(j, data=origin[i][j][:])

In [7]:
t0 = time.time()
out = convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc', missing=[38])
print("Time elapsed: ", time.time()-t0, "s")

loading data
loading complete
converting to 38
[36, 39]
with 36
with 39
Time elapsed:  11.041162967681885 s


In [8]:
out[0]

,adjustment_id,advanced_assimilation_feedback,advanced_homogenisation,advanced_qc,advanced_uncertainty,bbox_max_latitude,bbox_max_longitude,bbox_min_latitude,bbox_min_longitude,code_table,...,sensor_automation_status,sensor_id,source_id,spatial_representativeness,traceability,units,value_significance,z_coordinate,z_coordinate_method,z_coordinate_type
5124,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,70000.0,-2147483648,1
5125,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,70900.0,-2147483648,1
5126,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,74600.0,-2147483648,1
5127,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,75000.0,-2147483648,1
5128,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,79600.0,-2147483648,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10243,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,95000.0,-2147483648,1
10244,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,100000.0,-2147483648,1
10245,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,101100.0,-2147483648,1
4310,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,5,-2147483648,80000.0,-2147483648,1


In [9]:
out[0].observed_variable

5124     38
5125     38
5126     38
5127     38
5128     38
         ..
10243    38
10244    38
10245    38
4310     38
4311     38
Name: observed_variable, Length: 5124, dtype: int64

In [10]:
out[0].observation_value

5124     0.152794
5125     0.151932
5126     0.141759
5127     0.152478
5128     0.152282
           ...   
10243    0.574890
10244    0.484368
10245    0.435136
4310     0.234856
4311     0.225538
Name: observation_value, Length: 5124, dtype: float32

In [19]:
# sort it, so the recordindex will be correct

In [18]:
out[0].sort_values(by=['report_id','z_coordinate'], inplace=True)
out[0]

,adjustment_id,advanced_assimilation_feedback,advanced_homogenisation,advanced_qc,advanced_uncertainty,bbox_max_latitude,bbox_max_longitude,bbox_min_latitude,bbox_min_longitude,code_table,...,sensor_automation_status,sensor_id,source_id,spatial_representativeness,traceability,units,value_significance,z_coordinate,z_coordinate_method,z_coordinate_type
5124,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,70000.0,-2147483648,1
5125,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,70900.0,-2147483648,1
5126,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,74600.0,-2147483648,1
5127,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,75000.0,-2147483648,1
5128,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,79600.0,-2147483648,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10241,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,89400.0,-2147483648,1
10242,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,90000.0,-2147483648,1
10243,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,95000.0,-2147483648,1
10244,NaN,1,-2147483648,-2147483648,-2147483648,NaN,NaN,NaN,NaN,-2147483648,...,-2147483648,NA,era5_2,-2147483648,-2147483648,0,-2147483648,100000.0,-2147483648,1


In [72]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    print(data.observations_table)


observations_table:

adjustment_id_____________________________________ : : (41144,)
advanced_assimilation_feedback____________________ : : (41144,)
advanced_homogenisation___________________________ : : (41144,)
advanced_qc_______________________________________ : : (41144,)
advanced_uncertainty______________________________ : : (41144,)
bbox_max_latitude_________________________________ : : (41144,)
bbox_max_longitude________________________________ : : (41144,)
bbox_min_latitude_________________________________ : : (41144,)
bbox_min_longitude________________________________ : : (41144,)
code_table________________________________________ : : (41144,)
conversion_flag___________________________________ : : (41144,)
conversion_method_________________________________ : : (41144,)
crs_______________________________________________ : : (41144,)
data_policy_licence_______________________________ : : (41144,)
date_time_________________________________________ : : (41144,)
date_time_meaning__

In [167]:
import netCDF4

In [45]:
with netCDF4.Dataset('/jetfs/home/uvoggenberger/CEUAS/CEUAS/public/resort/converted/0-20000-0-11019_CEUAS_merged_v0.nc') as test:
    print(test.recordinidces)

NameError: name 'netCDF4' is not defined

In [52]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    print(data.recordindices.keys())
    print(data.recordindices['36'][:])

['104', '105', '106', '107', '117', '136', '137', '138', '139', '140', '36', '38', '39', '85', 'index', 'recordtimestamp', 'shape']
[   0   18   36   57   66   84   98  112  123  134  151  163  180  194
  205  221  240  261  266  277  295  308  327  341  354  367  381  401
  418  435  448  461  484  505  523  542  557  576  588  603  623  642
  658  676  691  711  733  749  767  781  795  812  826  841  859  876
  893  911  932  952  972  993 1010 1031 1050 1068 1075 1084 1097 1115
 1125 1137 1151 1165 1176 1188 1198 1211 1228 1246 1266 1279 1294 1307
 1330 1344 1364 1378 1396 1416 1440 1456 1474 1495 1513 1529 1547 1555
 1573 1589 1600 1607 1615 1633 1647 1653 1669 1687 1709 1727 1745 1766
 1772 1783 1796 1817 1833 1850 1867 1880 1895 1910 1933 1953 1972 1994
 2014 2032 2050 2068 2083 2100 2114 2129 2144 2158 2174 2189 2209 2223
 2237 2256 2275 2290 2308 2328 2344 2363 2378 2395 2410 2436 2451 2463
 2478 2498 2514 2526 2542 2549 2563 2580 2597 2617 2635 2653 2670 2685
 2702 2719 2733 

In [101]:
t0 = time.time()
out = convert_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc', missing=[38])
print("Time elapsed: ", time.time()-t0, "s")

loading data
loading complete
converting to 38
[36, 39]
with 36
with 39
Time elapsed:  5.298582077026367 s


In [77]:
check_for_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc')

[85, 38, 106, 107] 85
[38, 106, 107] 38
[106, 107] 106
[107] 107


[85, 38, 106, 107]

In [79]:
hums = [34, 38, 37, 36]
missing_hum_vars = check_for_missing('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc', checkvars = hums)
for i in missing_hum_vars: hums.remove(i)
print(hums)

[34, 38, 37, 36] 34
[38, 37, 36] 38
[37, 36] 37
[36] 36
[]


In [80]:
missing_hum_vars

[34, 38, 37, 36]

In [19]:
out[1]

,observed_variable,observation_value,date_time,z_coordinate,report_id,latitude,longitude
20817,107,2.0,1972-10-31 15:00:00,75000.0,20000000001,45.529999,-122.68
20818,107,2.0,1972-10-31 15:00:00,79700.0,20000000001,45.529999,-122.68
20819,107,2.0,1972-10-31 15:00:00,80000.0,20000000001,45.529999,-122.68
20820,107,2.0,1972-10-31 15:00:00,83500.0,20000000001,45.529999,-122.68
20821,107,1.0,1972-10-31 15:00:00,84400.0,20000000001,45.529999,-122.68
...,...,...,...,...,...,...,...
23985,107,2.0,1973-06-18 19:00:00,89400.0,20000000311,45.529999,-122.68
23986,107,2.0,1973-06-18 19:00:00,90000.0,20000000311,45.529999,-122.68
23987,107,2.0,1973-06-18 19:00:00,95000.0,20000000311,45.529999,-122.68
23988,107,2.0,1973-06-18 19:00:00,100000.0,20000000311,45.529999,-122.68


In [81]:
comb = out[0].append(out[1], ignore_index=True)
comb.drop_duplicates(('report_id', 'z_coordinate'))


,observed_variable,observation_value,date_time,z_coordinate,report_id,latitude,longitude
0,38,0.153335,1972-10-30 14:00:00,70000.0,20000000000,45.529999,-122.68
1,38,0.152464,1972-10-30 14:00:00,70900.0,20000000000,45.529999,-122.68
2,38,0.142243,1972-10-30 14:00:00,74600.0,20000000000,45.529999,-122.68
3,38,0.152946,1972-10-30 14:00:00,75000.0,20000000000,45.529999,-122.68
4,38,0.152703,1972-10-30 14:00:00,79600.0,20000000000,45.529999,-122.68
...,...,...,...,...,...,...,...
5119,38,0.680769,1973-06-18 19:00:00,89400.0,20000000311,45.529999,-122.68
5120,38,0.662534,1973-06-18 19:00:00,90000.0,20000000311,45.529999,-122.68
5121,38,0.574755,1973-06-18 19:00:00,95000.0,20000000311,45.529999,-122.68
5122,38,0.484200,1973-06-18 19:00:00,100000.0,20000000311,45.529999,-122.68


In [ ]:
df.drop_duplicates(subset=['brand', 'style'], keep='last')

In [28]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    print(data.recordindices['85'][-1]-1 - data.recordindices['85'][0])
    print(data.observations_table)

5136
observations_table:

adjustment_id_____________________________________ : : (41144,)
advanced_assimilation_feedback____________________ : : (41144,)
advanced_homogenisation___________________________ : : (41144,)
advanced_qc_______________________________________ : : (41144,)
advanced_uncertainty______________________________ : : (41144,)
bbox_max_latitude_________________________________ : : (41144,)
bbox_max_longitude________________________________ : : (41144,)
bbox_min_latitude_________________________________ : : (41144,)
bbox_min_longitude________________________________ : : (41144,)
code_table________________________________________ : : (41144,)
conversion_flag___________________________________ : : (41144,)
conversion_method_________________________________ : : (41144,)
crs_______________________________________________ : : (41144,)
data_policy_licence_______________________________ : : (41144,)
date_time_________________________________________ : : (41144,)
date_time_mean

In [71]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'date_time', 'z_coordinate', 'report_id', 'latitude', 'longitude'])
    print(data.recordindices['85'][-1] - data.recordindices['85'][0])
    print(data.recordindices['38'][-1] - data.recordindices['38'][0])
    print(data.recordindices['104'][-1] - data.recordindices['104'][0])
    print(data.recordindices['105'][-1] - data.recordindices['105'][0])
    
t = df[df.observed_variable == 85]
hum = df[df.observed_variable == 38]

cleaned_t = t[0:0]
cleaned_hum = hum[0:0]
for i in hum.report_id.drop_duplicates():
    inter_t = t[t.report_id == i]
    inter_hum = hum[hum.report_id == i]
    cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
    cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

t = cleaned_t.observation_value.to_xarray()
p = cleaned_t.z_coordinate.to_xarray()
rh = cleaned_hum.observation_value.to_xarray()

rh = rasotools.met.convert.to_sh(temp=t, press=p, rel_humi=rh)
out = cleaned_t.copy()
out['observed_variable'] =  38 
out['observation_value'] =  rh
print(out)
#             out['observation_value'] = np.asarray(rh)


5137
5122
3173
3173
       observed_variable  observation_value           date_time  z_coordinate  \
15680                 38           0.000783 1972-10-30 14:00:00       70000.0   
15681                 38           0.000780 1972-10-30 14:00:00       70900.0   
15682                 38           0.000755 1972-10-30 14:00:00       74600.0   
15683                 38           0.000813 1972-10-30 14:00:00       75000.0   
15684                 38           0.000822 1972-10-30 14:00:00       79600.0   
...                  ...                ...                 ...           ...   
20812                 38           0.004203 1973-06-18 19:00:00       89400.0   
20813                 38           0.004236 1973-06-18 19:00:00       90000.0   
20814                 38           0.004638 1973-06-18 19:00:00       95000.0   
20815                 38           0.004835 1973-06-18 19:00:00      100000.0   
20816                 38           0.004583 1973-06-18 19:00:00      101100.0   

       

In [ ]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11035_CEUAS_merged_v0.nc') as data:
    df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'date_time', 'z_coordinate', 'report_id', 'latitude', 'longitude'])
wind = df[df.observed_variable == 106]
wind = wind.append(df[df.observed_variable == 107])
errorrep = []
errorlev = []
for i in wind.report_id:
    dat1 = wind[wind.report_id == i] 
    for j in dat1.z_coordinate:
        dat2 = dat1[dat1.z_coordinate == j]
        wdir = df[df.observed_variable == 106]
        wspe = df[df.observed_variable == 107]
        if len(wdir) != len(wspe):
            errorrep.append(i)
            errorlev.append(j)


In [21]:
with eua.CDMDataset('/raid60/scratch/uli/resorted_files_correct/0-20000-0-11019_CEUAS_merged_v0.nc') as data:
    ri_t = data.recordindices['85'][:]
#     ri_34 = data.recordindices['34'][:]
    ri_36 = data.recordindices['36'][:]
    ri_38 = data.recordindices['38'][:]
    ri_39 = data.recordindices['39'][:]
    obsvar = data.observations_table.observed_variable[:]
    obsval = data.observations_table.observation_value[:]
    pres = data.observations_table.z_coordinate[:]
    repid = data.observations_table.report_id[:]
    
t = obsval[ri_t[0]:ri_t[-1]-1]
trep = repid[ri_t[0]:ri_t[-1]-1]
p_t = pres[ri_t[0]:ri_t[-1]-1]
hum39 = obsval[ri_39[0]:ri_39[-1]-1]
hum39rep = repid[ri_39[0]:ri_39[-1]-1]
p_hum39 = pres[ri_39[0]:ri_39[-1]-1]

In [78]:
# from numba import jit
# @jit(nopython=True) 
def get_to_same_len(a, b, repa, repb, pa, pb):
#     anew = a[np.in1d(repa, np.unique(repb))]
#     panew = pa[np.in1d(repa, np.unique(repb))]
#     bnew = b[np.in1d(repb, np.unique(repa))]
#     pbnew = pb[np.in1d(repb, np.unique(repa))]
    bout = np.empty(0)
    pbout = np.empty(0)
    for i in np.unique(repa[np.in1d(repa, np.unique(repb))]):
        bout = np.concatenate((bout, bnew[np.in1d(repb, i)]))
        pbout = np.concatenate((pbout, pbnew[np.in1d(repb, i)]))
    aout = np.empty(0)
    paout = np.empty(0)
    for i in np.unique(repb[np.in1d(repb, np.unique(repa))]):
        aout = np.concatenate((aout, anew[np.in1d(repa, i)]))
        paout = np.concatenate((paout, panew[np.in1d(repa, i)]))
    return aout, bout, paout, pbout

In [83]:
trep_dc = np.empty(len(trep))
for i in range(len(trep)):
    trep_dc[i] = (int(b"".join(trep[i]).decode()))
hum39rep_dc = np.empty(len(hum39rep))
for i in range(len(hum39rep)):
    hum39rep_dc[i] = (int(b"".join(hum39rep[i]).decode()))

In [84]:
a, b, pa, pb = get_to_same_len(t, hum39, trep_dc, hum39rep_dc, p_t, p_hum39)

In [85]:
print(len(a))
print(len(b))
print(len(pa))
print(len(pb))

0
5168
0
5168


In [177]:
tneu = t[np.in1d(trep_dc, np.unique(hum39rep_dc))]
p_tneu = p_t[np.in1d(trep_dc, np.unique(hum39rep_dc))]
hum39neu = hum39[np.in1d(hum39rep_dc, np.unique(trep_dc))]
p_hum39neu = p_hum39[np.in1d(hum39rep_dc, np.unique(trep_dc))]
hum39out = []
p_hum39out = []
for i in np.unique(trep_dc[np.in1d(trep_dc, np.unique(hum39rep_dc))]):
    hum39out = hum39out + (list(hum39neu[np.in1d(hum39rep_dc, i)]))
    p_hum39out = p_hum39out + (list(p_hum39neu[np.in1d(hum39rep_dc, i)]))
tout = []
p_tout = []
for i in np.unique(hum39rep_dc[np.in1d(hum39rep_dc, np.unique(trep_dc))]):
    tout = tout + (list(tneu[np.in1d(trep_dc, i)]))
    p_tout = p_tout + (list(p_tneu[np.in1d(trep_dc, i)]))


In [179]:
len(p_tout)

5136

In [180]:
len(p_hum39out)

5433

In [114]:
len(t[mask])

3321027

In [115]:
len(hum39)

2819680

In [ ]:
test = np.array([0, 1, 2, 5, 0])

states = [0, 2]

mask = np.in1d(test, states)

mask
array([ True, False,  True, False,  True])

test[mask]
array([0, 2, 0])

In [117]:
trep_dc = []
for i in trep:
    trep_dc.append(b"".join(i).decode())
hum39rep_dc = []
for i in hum39rep:
    hum39rep_dc.append(b"".join(i).decode())

In [101]:
len(np.unique(trep_dc))

50552

In [102]:
hum39rep_dc = []
for i in hum39rep:
    hum39rep_dc.append(b"".join(i).decode())

In [103]:
len(np.unique(hum39rep_dc))

38442

In [108]:
np.where(np.asarray(hum39rep_dc).isin(np.unique(trep_dc)))

AttributeError: 'numpy.ndarray' object has no attribute 'isin'

In [54]:
rh

<xarray.DataArray 'rh' (index: 5122)>
array([0.152205, 0.151341, 0.141188, ..., 0.572029, 0.481809, 0.432807],
      dtype=float32)
Coordinates:
  * index    (index) int64 15680 15681 15682 15683 ... 20813 20814 20815 20816
Attributes:
    units:               1
    standard_name:       relativ_humidity
    long_name:           relative humidity
    esat:                HylandWexler
    origin:              t,q,p
    enhancement_factor:  yes
    precision:           6

In [52]:
sh

,observed_variable,observation_value,date_time,z_coordinate,report_id,latitude,longitude
10246,39,0.000780,1972-10-30 14:00:00,70000.0,20000000000,45.529999,-122.68
10247,39,0.000777,1972-10-30 14:00:00,70900.0,20000000000,45.529999,-122.68
10248,39,0.000752,1972-10-30 14:00:00,74600.0,20000000000,45.529999,-122.68
10249,39,0.000810,1972-10-30 14:00:00,75000.0,20000000000,45.529999,-122.68
10250,39,0.000819,1972-10-30 14:00:00,79600.0,20000000000,45.529999,-122.68
...,...,...,...,...,...,...,...
15674,39,0.004183,1973-06-18 19:00:00,89400.0,20000000311,45.529999,-122.68
15675,39,0.004216,1973-06-18 19:00:00,90000.0,20000000311,45.529999,-122.68
15676,39,0.004615,1973-06-18 19:00:00,95000.0,20000000311,45.529999,-122.68
15677,39,0.004809,1973-06-18 19:00:00,100000.0,20000000311,45.529999,-122.68


In [ ]:
sh[]

In [ ]:
# #
# # todo: 
# # variablen nur einmal lesen, und nicht für jedern convertierweg nochmal
# # für alle methoden sammeln und dann das, was mehrfach da ist, aussortieren
# # das übrige dann mit dem vergleichen, was schon im Datensatz steht -> auffüllen 
# # fertig
# #



# def convert_missing(file, missing, destination: str = './converted'):
#     with eua.CDMDataset(file) as data:
#         df = data.to_dataframe(groups='observations_table', variables=['observed_variable', 'observation_value', 'date_time', 'z_coordinate', 'report_id', 'latitude', 'longitude'])
        
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     if 106 in missing or 107 in missing: # wd and/or ws is missing -> calculate it from u/v
#         u = np.asarray(df[df.observed_variable == 104].observation_value)
#         v = np.asarray(df[df.observed_variable == 105].observation_value)
#         ws = np.sqrt(u ** 2 + v ** 2)
#         wd = 90 - np.arctan2(-v, -u) * 180 / np.pi - 180.
#         wd = np.where(wd > 0., wd, 360.+wd)
#         return wd, ws
#         #add
#         #set conversion flag
#     if 104 in missing or 105 in missing: # u and/or v is missing -> calculate it from wd and ws
#         wd = np.asarray(df[df.observed_variable == 106].observation_value)
#         ws = np.asarray(df[df.observed_variable == 107].observation_value)
#         u = ws * np.cos(np.radians(wd))
#         v = ws * np.sin(np.radians(wd)) 
#         return u, v
#         #add
#         #set conversion flag
            
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     #
#     # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
#     #
#     if 38 in missing: # rh is missing -> calculate it from 
#         hums = [34, 36, 37, 39]
#         missing_hum_vars = check_for_missing(file, checkvars = hums)
#         for i in missing_hum_vars: hums.remove(i)
#         print(hums)
# #         if 34 in hums: # dew point depression
# #             print('with 34')
# #             t = df[df.observed_variable == 85]
# #             hum = df[df.observed_variable == 34]
            
# #             cleaned_t = t[0:0]
# #             cleaned_hum = hum[0:0]
# #             for i in hum.report_id.drop_duplicates():
# #                 inter_t = t[t.report_id == i]
# #                 inter_hum = hum[hum.report_id == i]
# #                 cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
# #                 cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])
        
# #             t = cleaned_t.observation_value.to_xarray()
# #             dpd = cleaned_hum.observation_value.to_xarray()

# #             rh = rasotools.met.convert.to_rh(t, dpd=dpd)
# #             return rh
# #             #add
# #             #set conversion flag
#         if 36 in hums: # dew point
#             print('with 36')
#             t = df[df.observed_variable == 85]
#             hum = df[df.observed_variable == 36]

#             cleaned_t = t[0:0]
#             cleaned_hum = hum[0:0]
#             for i in hum.report_id.drop_duplicates():
#                 inter_t = t[t.report_id == i]
#                 inter_hum = hum[hum.report_id == i]
#                 cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
#                 cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

#             dpd = cleaned_t.observation_value.subtract(np.asarray(cleaned_hum.observation_value))
#             t = cleaned_t.observation_value.to_xarray()
#             dpd = dpd.to_xarray()

#             rh = rasotools.met.convert.to_rh(t,dpd=dpd)
#             return rh
#             #add
#             #set conversion flag
#         if 39 in hums: # specific humidity
#             print('with 39')
#             t = df[df.observed_variable == 85]
#             hum = df[df.observed_variable == 39]

#             cleaned_t = t[0:0]
#             cleaned_hum = hum[0:0]
#             for i in hum.report_id.drop_duplicates():
#                 inter_t = t[t.report_id == i]
#                 inter_hum = hum[hum.report_id == i]
#                 cleaned_t = cleaned_t.append(inter_t[inter_t.z_coordinate.isin(inter_hum.z_coordinate)])
#                 cleaned_hum = cleaned_hum.append(inter_hum[inter_hum.z_coordinate.isin(inter_t.z_coordinate)])

#             p = cleaned_t.z_coordinate.to_xarray()
#             t = cleaned_t.observation_value.to_xarray()
#             sh = cleaned_hum.observation_value.to_xarray()

#             rh = rasotools.met.convert.to_rh(temp=t, spec_humi=sh, press=p)
#             return rh
#             #add
#             #set conversion flag
#         else:
#             print('-')
#             print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
#             print('-')
            
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     #
#     # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
#     #
#     if 34 in missing: # dew point depression is missing -> calculate it from
#         hums = [38, 36, 37, 39]
#         missing_hum_vars = check_for_missing(file, checkvars = hums)
#         for i in missing_hum_vars: hums.remove(i)
            
#         if 38 in hums and not 85 in missing: # relative humidity
#             t = df[df.observed_variable == 85]
#             rh = df[df.observed_variable == 38]

#             cleaned = t[0:0]
#             for i in rh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in rh[rh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             rh = rh.observation_value.to_xarray()

#             dpd = rasotools.met.convert.to_dpd(temp=t,press=p,rel_humi=rh)
#             return dpd
#             #add
#             #set conversion flag
#         if 36 in hums: # dew point
#             t = df[df.observed_variable == 85]
#             dp = df[df.observed_variable == 36]

#             cleaned = t[0:0]
#             for i in dp.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in dp[dp.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             dpd = cleaned.observation_value.subtract(np.asarray(dp.observation_value))
#             dpd = dpd.to_xarray()
#             return dpd
#             #add
#             #set conversion flag
#         if 39 in hums: # specific humidity
#             t = df[df.observed_variable == 85]
#             sh = df[df.observed_variable == 39]

#             cleaned = t[0:0]
#             for i in sh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in sh[sh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             p = cleaned.z_coordinate.to_xarray()
#             t = cleaned.observation_value.to_xarray()
#             sh = sh.observation_value.to_xarray()

#             dpd = rasotools.met.convert.to_dpd(spec_humi=sh,press=p,temp=t)
#             return dpd
#             #add
#             #set conversion flag
#         else:
#             print('-')
#             print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
#             print('-')
            
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     #
#     # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
#     #
#     if 36 in missing: # dew point is missing -> calculate it from
#         hums = [34, 38, 37, 39]
#         missing_hum_vars = check_for_missing(file, checkvars = hums)
#         for i in missing_hum_vars: hums.remove(i)
            
#         if 38 in hums and not 85 in missing: # relative humidity
#             t = df[df.observed_variable == 85]
#             rh = df[df.observed_variable == 38]

#             cleaned = t[0:0]
#             for i in rh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in rh[rh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             rh = rh.observation_value.to_xarray()

#             dp = rasotools.met.convert.to_dewpoint(temp=t,press=p,rel_humi=rh)
#             return dp
#             #add
#             #set conversion flag
#         if 34 in hums: # dew point depression
#             t = df[df.observed_variable == 85]
#             dpd = df[df.observed_variable == 34]

#             cleaned = t[0:0]
#             for i in dpd.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 cleaned.append(inter)
# #                 for j in dpd[dpd.report_id == i].z_coordinate:
# #                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             dp = cleaned.observation_value.subtract(np.asarray(dpd.observation_value))
#             dp = dp.to_xarray()
#             return dp
#             #add
#             #set conversion flag
#         if 39 in hums: # specific humidity
#             t = df[df.observed_variable == 85]
#             sh = df[df.observed_variable == 39]

#             cleaned = t[0:0]
#             for i in sh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in sh[sh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             p = cleaned.z_coordinate.to_xarray()
#             sh = sh.observation_value.to_xarray()
#             dp = rasotools.met.convert.to_dewpoint(spec_humi=sh,press=p)
#             return dp
#             #add
#             #set conversion flag
#         else:
#             print('-')
#             print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
#             print('-')
            
# # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#     #
#     # convert from every source and merge the rh outputs? maximize the searched variable -> removed elif and made if everywher
#     #
#     if 39 in missing: # specific humidity is missing -> calculate it from
#         hums = [34, 38, 37, 36]
#         missing_hum_vars = check_for_missing(file, checkvars = hums)
#         for i in missing_hum_vars: hums.remove(i)
            
#         if 38 in hums and not 85 in missing: # relative humidity
#             t = df[df.observed_variable == 85]
#             rh = df[df.observed_variable == 38]

#             cleaned = t[0:0]
#             for i in rh.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in rh[rh.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             rh = rh.observation_value.to_xarray()

#             sh = rasotools.met.convert.to_sh(temp=t, press=p, rel_humi=rh)
#             return sh
#             #add
#             #set conversion flag
#         if 34 in hums: # dew point depression
#             t = df[df.observed_variable == 85]
#             dpd = df[df.observed_variable == 34]

#             cleaned = t[0:0]
#             for i in dpd.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in dpd[dpd.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             dpd = dpd.observation_value.to_xarray()

#             sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
#             return sh
#             #add
#             #set conversion flag
#         if 36 in hums: # dew point
#             t = df[df.observed_variable == 85]
#             dp = df[df.observed_variable == 36]

#             cleaned = t[0:0]
#             for i in dp.report_id.drop_duplicates():
#                 inter = t[t.report_id == i]
#                 for j in dp[dp.report_id == i].z_coordinate:
#                     cleaned = cleaned.append(inter[inter.z_coordinate == j])

#             dpd = cleaned.observation_value.subtract(np.asarray(dp.observation_value))

#             t = cleaned.observation_value.to_xarray()
#             p = cleaned.z_coordinate.to_xarray()
#             dpd = dpd.to_xarray()
#             sh = rasotools.met.convert.to_sh(dpd=dpd, press=p, temp=t)
#             return sh
#             #add
#             #set conversion flag
#         else:
#             print('-')
#             print('NO HUMIDITY VARIABLE AVAILABLE - COULD NOT CONVERT')
#             print('-')